In [16]:
#imports
import config
import requests
import json
import csv
import pprint
import pandas as pd
import time


# pretty printer for nicer formating when needed
pp = pprint.PrettyPrinter(indent = 1)


In [17]:
# Here we define our query as a multi-line string
query = '''
query ($id: Int, $page: Int, $perPage: Int, $search: String) {
    Page (page: $page, perPage: $perPage) {
        pageInfo {
            total
            currentPage
            lastPage
            hasNextPage
            perPage
        }
        media (id: $id, search: $search) {
            id
            title {
                romaji
                english
                native
            }
            startDate {
                year
                month
                day
            }
            description
            countryOfOrigin
            averageScore
            genres
        }
    }
}
'''
# 1571 pages with 50 per page
variables = {
    'page': 1,
    'perPage' : 50
}

url = 'https://graphql.anilist.co'

In [ ]:
response = pd.DataFrame((requests.post(url, json={'query': query, 'variables': variables}).json())['data']['Page']['media'])

In [18]:
anime = pd.DataFrame()

for i in range(1, 200):
    variables['page'] = i
    try:
        temp = pd.DataFrame((requests.post(url, json = {'query': query, 'variables': variables}).json())['data']['Page']['media'])
        anime = pd.concat([anime, temp], ignore_index = True)
    except:
        time.sleep(120)
        temp = pd.DataFrame((requests.post(url, json = {'query': query, 'variables': variables}).json())['data']['Page']['media'])
        anime = pd.concat([anime, temp], ignore_index = True)

In [19]:
for index, row in anime.iterrows():
    # changing format of date data
    month = '0'
    day = '0'
    year = '0'
    try:
        if row['startDate']['month'] < 10:
            month = '0' + str(row['startDate']['month'])
        else:
            month = str(row['startDate']['month'])
    except:
        pass

    try:
        if row['startDate']['day'] < 10:
            day = '0' + str(row['startDate']['day'])
        else:
            day = str(row['startDate']['day'])
    except:
        pass

    date = f"{str(row['startDate']['year'])}-{month}-{day}"
    anime.loc[index, 'first_air_date'] = date

    # changing format of name data
    name = row['title']['romaji']
    original_name = row['title']['native']

    anime.loc[index, 'name'] = name
    anime.loc[index, 'original_name'] = original_name

    if 'JP' not in row["countryOfOrigin"]:
        anime.drop(index, inplace = True)

anime = anime.rename({"description" : "overview", "countryOfOrigin" : "origin_country", "averageScore": "vote_average"}, axis = 'columns')
anime = anime.drop(["title", "startDate"], axis = 1)


In [20]:
cols = ['first_air_date', 'genres', 'id', 'name', 'origin_country', 'original_name', 'overview','vote_average']
anime = anime[cols]

In [22]:
anime.to_excel("anilist_test.xlsx", index = False)